In [1]:
from google.colab import files
uploaded = files.upload()

Saving protocols.csv to protocols.csv


In [2]:
!pip install tqdm --quiet

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import csv

# Mapping from network -> (verified contracts page URL, API key)
EXPLORER_CONFIGS = {
    'ethereum': {
        'contracts_page': 'https://etherscan.io/contractsVerified',
        'api_key': 'VZFDUWB3YGQ1YCDKTCU1D6DDSS',
        'api_url': 'https://api.etherscan.io/api'
    },
    'bsc': {
        'contracts_page': 'https://bscscan.com/contractsVerified',
        'api_key': 'ZM8ACMJB67C2IXKKBF8URFUNSY',
        'api_url': 'https://api.bscscan.com/api'
    },
    'avalanche': {
        'contracts_page': 'https://snowtrace.io/contractsVerified',
        'api_key': 'ATJQERBKV1CI3GVKNSE3Q7RGEJ',
        'api_url': 'https://api.snowtrace.io/api'
    },
    'arbitrum': {
        'contracts_page': 'https://arbiscan.io/contractsVerified',
        'api_key': 'B6SVGA7K3YBJEQ69AFKJF4YHVX',
        'api_url': 'https://api.arbiscan.io/api'
    },
    'optimism': {
        'contracts_page': 'https://optimistic.etherscan.io/contractsVerified',
        'api_key': '66N5FRNV1ZD4I87S7MAHCJVXFJ',
        'api_url': 'https://api-optimistic.etherscan.io/api'
    }
}

def fetch_verified_contracts_by_keyword(keyword, explorer_url, pages=3):
    contracts = []
    for page in range(1, pages+1):
        params = {'p': page}
        print(f"Scraping page {page} on {explorer_url} for keyword {keyword}")
        response = requests.get(explorer_url, params=params)
        if response.status_code != 200:
            print(f"Failed to fetch page {page} on {explorer_url}")
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'table table-md-text-normal table-hover'})
        if not table:
            print("No contracts table found, stopping.")
            break
        rows = table.find_all('tr')[1:]  # skip header
        for row in rows:
            cols = row.find_all('td')
            if len(cols) < 5:
                continue
            contract_address = cols[1].text.strip()
            contract_name = cols[2].text.strip()
            description = cols[3].text.strip().lower()
            if keyword.lower() in contract_name.lower() or keyword.lower() in description:
                contracts.append({
                    'contract_address': contract_address,
                    'contract_name': contract_name
                })
        time.sleep(1)  # polite delay to avoid rate limits
    return contracts

def main():
    df = pd.read_csv('protocols.csv')
    unique_pairs = df[['protocol', 'network']].drop_duplicates()
    all_contracts = []
    for idx, row in unique_pairs.iterrows():
        protocol = row['protocol']
        network = row['network'].lower()
        print(f"Processing {protocol} on {network}")
        if network in EXPLORER_CONFIGS:
            conf = EXPLORER_CONFIGS[network]
            contracts = fetch_verified_contracts_by_keyword(
                protocol, conf['contracts_page'], pages=3)
            for c in contracts:
                all_contracts.append({
                    'protocol': protocol,
                    'network': network,
                    'contract_address': c['contract_address'],
                    'contract_name': c['contract_name']
                })
        else:
            print(f"No scraper configured for network: {network}")
    # Save combined results
    if all_contracts:
        keys = all_contracts[0].keys()
        with open('all_protocols_contracts.csv', 'w', newline='') as f:
            dict_writer = csv.DictWriter(f, fieldnames=keys)
            dict_writer.writeheader()
            dict_writer.writerows(all_contracts)
        print(f"Saved {len(all_contracts)} contracts in all_protocols_contracts.csv")
    else:
        print("No contracts found.")

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
No scraper configured for network: sanko
Processing camelot_v3 on sanko
No scraper configured for network: sanko
Processing sudoswap_v2 on sanko
No scraper configured for network: sanko
Processing bitusd on sapphire
No scraper configured for network: sapphire
Processing vine_money on sapphire
No scraper configured for network: sapphire
Processing illuminex on sapphire
No scraper configured for network: sapphire
Processing neby_dex on sapphire
No scraper configured for network: sapphire
Processing thorn_protocol on sapphire
No scraper configured for network: sapphire
Processing rosy_stake on sapphire
No scraper configured for network: sapphire
Processing accumulated_finance_lending on sapphire
No scraper configured for network: sapphire
Processing accumulated_finance_liquid_staking on sapphire
No scraper configured for network: sapphire
Processing midas_rwa on sapphire
No scraper configured for network: sapphire
Processing savmswap on s

In [5]:
import requests
import json

url = "https://api.llama.fi/protocols"
response = requests.get(url)
if response.status_code == 200:
    protocols = response.json()
    with open('llama_protocols.json', 'w') as f:
        json.dump(protocols, f, indent=2)
    print("Saved protocol data to llama_protocols.json")
else:
    print(f"Failed to fetch protocols data: {response.status_code}")

Saved protocol data to llama_protocols.json


In [6]:
import json
import csv

# Load your locally saved DeFiLlama protocols JSON file
with open('llama_protocols.json', 'r') as f:
    protocols = json.load(f)

results = []

for proto in protocols:
    protocol_name = proto.get('name', 'Unknown')
    protocol_slug = proto.get('slug', '')
    addr = proto.get('address')
    addresses = proto.get('addresses', {})

    # Collect all addresses (could be dict of chain->address(es))
    contracts = []

    if addr:
        contracts.append(addr)
    if addresses:
        for k, v in addresses.items():
            if isinstance(v, list):
                contracts.extend(v)
            elif isinstance(v, str):
                contracts.append(v)

    if contracts:
        # Deduplicate and clean up contract addresses
        uniq_contracts = list(set(contracts))
        for contract in uniq_contracts:
            results.append({
                'protocol': protocol_name,
                'slug': protocol_slug,
                'contract_address': contract
            })

# Save results to CSV
keys = ['protocol', 'slug', 'contract_address']
with open('defillama_extracted_contracts.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(results)

print(f"Extracted {len(results)} contract addresses to 'defillama_extracted_contracts.csv'")

# Optionally print some samples
for i, r in enumerate(results[:20]):
    print(r)

Extracted 3545 contract addresses to 'defillama_extracted_contracts.csv'
{'protocol': 'Aave V3', 'slug': 'aave-v3', 'contract_address': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9'}
{'protocol': 'Lido', 'slug': 'lido', 'contract_address': '0x5a98fcbea516cf06857215779fd812ca3bef1b32'}
{'protocol': 'EigenLayer', 'slug': 'eigenlayer', 'contract_address': '0xec53bf9167f50cdeb3ae105f56099aaab9061f83'}
{'protocol': 'Ethena USDe', 'slug': 'ethena-usde', 'contract_address': '0x57e114b691db790c35207b2e685d4a43181e6061'}
{'protocol': 'Pendle', 'slug': 'pendle', 'contract_address': '0x808507121b80c02388fad14726482e061b8da827'}
{'protocol': 'ether.fi Stake', 'slug': 'ether.fi-stake', 'contract_address': '0xfe0c30065b384f05761f15d0cc899d4f9f9cc0eb'}
{'protocol': 'Gate', 'slug': 'gate', 'contract_address': '0xe66747a101bff2dba3697199dcce5b743b454759'}
{'protocol': 'Morpho Blue', 'slug': 'morpho-blue', 'contract_address': '0x58D97B57BB95320F9a05dC918Aef65434969c2B2'}
{'protocol': 'Sky Lending', 'slug

In [9]:
import pandas as pd
import requests
import time
import csv
import re

# API configuration per network
API_CONFIG = {
    'ethereum': {
        'apikey': 'VZFDUWB3YGQ1YCDKTCU1D6DDSS',
        'base_url': 'https://api.etherscan.io/api'
    },
    'bsc': {
        'apikey': 'ZM8ACMJB67C2IXKKBF8URFUNSY',
        'base_url': 'https://api.bscscan.com/api'
    },
    'arbitrum': {
        'apikey': 'B6SVGA7K3YBJEQ69AFKJF4YHVX',
        'base_url': 'https://api.arbiscan.io/api'
    },
    'avalanche': {
        'apikey': 'ATJQERBKV1CI3GVKNSE3Q7RGEJ',
        'base_url': 'https://api.snowtrace.io/api'
    },
    'optimism': {
        'apikey': '66N5FRNV1ZD4I87S7MAHCJVXFJ',
        'base_url': 'https://api-optimistic.etherscan.io/api'
    }
}

# Function to extract network and address from contract_address string
def extract_network_and_address(contract_address):
    if isinstance(contract_address, str) and ':' in contract_address:
        parts = contract_address.split(':', 1)
        network = parts[0].lower()
        address = parts[1]
        return network, address
    return None, contract_address # Return None for network if no colon

def fetch_contract_abi(address, apikey, base_url):
    params = {
        'module': 'contract',
        'action': 'getabi',
        'address': address,
        'apikey': apikey
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if data.get('status') == '1':
        return data['result']
    return None

def fetch_contract_info(address, apikey, base_url):
    params = {
        'module': 'contract',
        'action': 'getsourcecode',
        'address': address,
        'apikey': apikey
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if data.get('status') == '1' and data.get('result'):
        return data['result'][0]  # dict with source code and contract name
    return None

input_csv = 'defillama_extracted_contracts.csv'
output_csv = 'contracts_with_abis.csv'

df = pd.read_csv(input_csv)
output = []

for idx, row in df.iterrows():
    protocol = row['protocol']
    original_contract_address = row['contract_address']

    # Check if contract_address is a string before processing
    if not isinstance(original_contract_address, str):
        print(f"Invalid contract address format for {protocol}: {original_contract_address}, skipping.")
        output.append({
            'protocol': protocol,
            'network': 'unknown',
            'contract_address': original_contract_address,
            'verified': 'no',
            'contract_name': None,
            'abi': None
        })
        continue

    network, contract_address = extract_network_and_address(original_contract_address)

    # If network was not extracted, try to infer from address format (basic check)
    if network is None:
        # This is a very basic attempt to infer network, might need refinement
        if re.match(r'^0x[a-fA-F0-9]{40}$', contract_address):
             # Assume Ethereum-like address format, could be ETH, BSC, Arbitrum, Optimism
             # For simplicity, we'll try the most common EVM chains
             possible_networks = ['ethereum', 'bsc', 'arbitrum', 'optimism']
             for possible_net in possible_networks:
                 if possible_net in API_CONFIG:
                    network = possible_net
                    break # Use the first matching network config found
        # Add more network inference logic here if needed for other address formats

    api_data = None
    if network and network in API_CONFIG:
         api_data = API_CONFIG.get(network)


    if not api_data:
        print(f"No API configured or network could not be inferred for {protocol} - {original_contract_address}, skipping.")
        output.append({
            'protocol': protocol,
            'network': network if network else 'unknown',
            'contract_address': original_contract_address,
            'verified': 'no',
            'contract_name': None,
            'abi': None
        })
        continue

    print(f"Fetching ABI for {protocol} - {network} - {contract_address}")
    abi = fetch_contract_abi(contract_address, api_data['apikey'], api_data['base_url'])
    source_info = fetch_contract_info(contract_address, api_data['apikey'], api_data['base_url'])
    verified = 'yes' if abi is not None else 'no'
    contract_name = None
    if source_info:
        contract_name = source_info.get('ContractName')

    output.append({
        'protocol': protocol,
        'network': network,
        'contract_address': original_contract_address,
        'verified': verified,
        'contract_name': contract_name,
        'abi': abi
    })

    # Respect API rate limits
    time.sleep(0.2)

# Save output CSV with ABI (ABI as string)
keys = ['protocol', 'network', 'contract_address', 'verified', 'contract_name', 'abi']
with open(output_csv, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(output)

print(f"Saved contract ABIs and info to {output_csv}")

Fetching ABI for Aave V3 - ethereum - 0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
Fetching ABI for Lido - ethereum - 0x5a98fcbea516cf06857215779fd812ca3bef1b32
Fetching ABI for EigenLayer - ethereum - 0xec53bf9167f50cdeb3ae105f56099aaab9061f83
Fetching ABI for Ethena USDe - ethereum - 0x57e114b691db790c35207b2e685d4a43181e6061
Fetching ABI for Pendle - ethereum - 0x808507121b80c02388fad14726482e061b8da827
Fetching ABI for ether.fi Stake - ethereum - 0xfe0c30065b384f05761f15d0cc899d4f9f9cc0eb
Fetching ABI for Gate - ethereum - 0xe66747a101bff2dba3697199dcce5b743b454759
Fetching ABI for Morpho Blue - ethereum - 0x58D97B57BB95320F9a05dC918Aef65434969c2B2
Fetching ABI for Sky Lending - ethereum - 0x56072c95faa701256059aa122697b133aded9279
Fetching ABI for Arbitrum Bridge - arbitrum - 0x912ce59144191c1204e64559fe8253a0e49e6548
No API configured or network could not be inferred for Hyperliquid Bridge - 0x0d01dc56dcaaca66ad901c959b4011ec, skipping.
Fetching ABI for Bitmex - ethereum - 0xb113c6

In [11]:
import pandas as pd
import json
import csv

input_csv = 'contracts_with_abis.csv'
output_csv = 'contracts_events.csv'

df = pd.read_csv(input_csv)
event_rows = []

def extract_event_signatures(abi_json_str):
    try:
        abi_json = json.loads(abi_json_str.replace("'", "\""))
        events = []
        for entry in abi_json:
            if entry.get('type') == 'event':
                name = entry.get('name', '')
                inputs = entry.get('inputs', [])
                arg_types = [f"{inp['type']} {inp['name']}" for inp in inputs]
                signature = f"{name}({', '.join(arg_types)})"
                events.append((name, signature))
        return events
    except Exception as e:
        return []

for _, row in df.iterrows():
    protocol = row['protocol']
    network = row.get('network', '')
    contract_address = row['contract_address']
    contract_name = row.get('contract_name', '')
    abi = row['abi']

    if pd.notnull(abi) and isinstance(abi, str) and len(abi) > 2:
        events = extract_event_signatures(abi)
        for event_name, event_sig in events:
            event_rows.append({
                'protocol': protocol,
                'network': network,
                'contract_address': contract_address,
                'contract_name': contract_name,
                'event_name': event_name,
                'event_signature': event_sig
            })

keys = ['protocol', 'network', 'contract_address', 'contract_name', 'event_name', 'event_signature']
with open(output_csv, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(event_rows)

print(f"Extracted events from ABIs and saved to {output_csv}")

Extracted events from ABIs and saved to contracts_events.csv


In [13]:
import pandas as pd
import requests
import time
import re

# API keys for the explorers
API_CONFIG = {
    'ethereum': {
        'apikey': '9UZDUCGM8JR5HRNIJ5732G9ERFV8US1GVR',
        'base_url': 'https://api.etherscan.io/api'
    },
    'bsc': {
        'apikey': 'FU496DJARA35GBUIA9D5MSEY24RNWXDSFA',
        'base_url': 'https://api.bscscan.com/api'
    },
    'avalanche': {
        'apikey': 'DD4YP7BDNI5MGXJP1RQ3G5C3WVNH6WW2E3',
        'base_url': 'https://api.snowtrace.io/api'
    },
    'arbitrum': {
        'apikey': 'B6SVGA7K3YBJEQ69AFKJF4YHVX',
        'base_url': 'https://api.arbiscan.io/api'
    },
    'optimism': {
        'apikey': '66N5FRNV1ZD4I87S7MAHCJVXFJ',
        'base_url': 'https://api-optimistic.etherscan.io/api'
    }
}


def extract_network_and_address(contract_address):
    if isinstance(contract_address, str) and ':' in contract_address:
        parts = contract_address.split(':', 1)
        network = parts[0].lower()
        address = parts[1]
        return network, address
    return None, contract_address # Return None for network if no colon


def fetch_abi(network, address, apikey, base_url):
    params = {
        'module': 'contract',
        'action': 'getabi',
        'address': address,
        'apikey': apikey
    }

    try:
        resp = requests.get(base_url, params=params)
        data = resp.json()
        if data['status'] == '1':
            return data['result']
        else:
            print(f"ABI not found for {address} on {network}: {data.get('result')}")
            return None
    except Exception as e:
        print(f"Exception for {address} on {network}: {e}")
        return None

# Load your contracts CSV (e.g. defillama_extracted_contracts.csv)
df = pd.read_csv('defillama_extracted_contracts.csv')

abis = []
for i, row in df.iterrows():
    original_contract_address = row['contract_address']

    # Check if contract_address is a string before processing
    if not isinstance(original_contract_address, str):
        print(f"Invalid contract address format: {original_contract_address}, skipping.")
        abis.append(None)
        continue

    network, contract_address = extract_network_and_address(original_contract_address)

    # If network was not extracted, try to infer from address format (basic check)
    if network is None:
        # This is a very basic attempt to infer network, might need refinement
        if re.match(r'^0x[a-fA-F0-9]{40}$', contract_address):
             # Assume Ethereum-like address format, could be ETH, BSC, Arbitrum, Optimism
             # For simplicity, we'll try the most common EVM chains
             possible_networks = ['ethereum', 'bsc', 'arbitrum', 'optimism']
             for possible_net in possible_networks:
                 if possible_net in API_CONFIG:
                    network = possible_net
                    break # Use the first matching network config found
        # Add more network inference logic here if needed for other address formats


    api_data = None
    if network and network in API_CONFIG:
         api_data = API_CONFIG.get(network)

    if not api_data:
        print(f"No API configured or network could not be inferred for {original_contract_address}, skipping.")
        abis.append(None)
        continue

    print(f"Fetching ABI for {contract_address} on {network}...")
    abi = fetch_abi(network, contract_address, api_data['apikey'], api_data['base_url'])
    abis.append(abi)

    # Avoid hammering API rate limits
    time.sleep(0.2)

# Attach the ABI column to dataframe
df['abi'] = abis
df['network'] = [extract_network_and_address(addr)[0] if isinstance(addr, str) else None for addr in df['contract_address']]


# Save the dataset with ABI included
df.to_csv('contracts_with_abis.csv', index=False)

print('Completed fetching ABIs and saved contracts_with_abis.csv')

Fetching ABI for 0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9 on ethereum...
Fetching ABI for 0x5a98fcbea516cf06857215779fd812ca3bef1b32 on ethereum...
Fetching ABI for 0xec53bf9167f50cdeb3ae105f56099aaab9061f83 on ethereum...
Fetching ABI for 0x57e114b691db790c35207b2e685d4a43181e6061 on ethereum...
Fetching ABI for 0x808507121b80c02388fad14726482e061b8da827 on ethereum...
Fetching ABI for 0xfe0c30065b384f05761f15d0cc899d4f9f9cc0eb on ethereum...
Fetching ABI for 0xe66747a101bff2dba3697199dcce5b743b454759 on ethereum...
Fetching ABI for 0x58D97B57BB95320F9a05dC918Aef65434969c2B2 on ethereum...
Fetching ABI for 0x56072c95faa701256059aa122697b133aded9279 on ethereum...
Fetching ABI for 0x912ce59144191c1204e64559fe8253a0e49e6548 on arbitrum...
ABI not found for 0x912ce59144191c1204e64559fe8253a0e49e6548 on arbitrum: You are using a deprecated V1 endpoint, switch to Etherscan API V2 using https://docs.etherscan.io/v2-migration
No API configured or network could not be inferred for 0x0d01dc5

In [14]:
import pandas as pd
import json
import csv

input_csv = 'contracts_with_abis.csv'
output_csv = 'contracts_events_and_functions.csv'

df = pd.read_csv(input_csv)
rows = []

def extract_signatures(abi_str):
    try:
        abi = json.loads(abi_str)
        events = []
        functions = []
        for entry in abi:
            typ = entry.get('type')
            name = entry.get('name', '')
            inputs = entry.get('inputs', [])
            args = ", ".join([f"{inp['type']} {inp['name']}" for inp in inputs])
            signature = f"{name}({args})"
            if typ == 'event':
                events.append(signature)
            elif typ == 'function':
                functions.append(signature)
        return events, functions
    except:
        return [], []

for _, row in df.iterrows():
    protocol = row.get('protocol', '')
    network = row.get('network', '')
    contract_address = row['contract_address']
    contract_name = row.get('contract_name', '')
    abi = row.get('abi', '')

    if pd.notnull(abi) and isinstance(abi, str) and len(abi) > 2:
        events, functions = extract_signatures(abi)
        for ev in events:
            rows.append({'protocol': protocol, 'network': network, 'contract_address': contract_address, 'contract_name': contract_name, 'type': 'event', 'signature': ev})
        for fn in functions:
            rows.append({'protocol': protocol, 'network': network, 'contract_address': contract_address, 'contract_name': contract_name, 'type': 'function', 'signature': fn})

keys = ['protocol', 'network', 'contract_address', 'contract_name', 'type', 'signature']

with open(output_csv, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(rows)

print(f"Extracted contract events and functions saved to {output_csv}")

Extracted contract events and functions saved to contracts_events_and_functions.csv


In [15]:
import pandas as pd
import json

# Load the contracts with ABI info (update path if needed!)
df = pd.read_csv('contracts_with_abis.csv')

output = []
for _, row in df.iterrows():
    item = {
        "protocol": row.get("protocol", ""),
        "address": row.get("contract_address", row.get("address", "")),
        "contract_name": row.get("contract_name", ""),
        "abi": row.get("abi", None)
    }
    output.append(item)

with open("all_protocols_contracts_with_abis.json", "w") as f:
    json.dump(output, f, indent=2)

print("Saved to all_protocols_contracts_with_abis.json")

Saved to all_protocols_contracts_with_abis.json
